In [ ]:
!pip install -q transformers sentencepiece

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "tsmatz/mt5_summarize_japanese"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def summarize(text: str, target_chars=120):
    """
    text : 入力テキスト
    target_chars : 「だいたいこのくらいの文字数で書いてほしい」という希望
    """
    # mT5 は「文字数」ではなく「トークン数」で制限される → だいたいでOK
    max_length = int(target_chars * 1.3)  # 少し余裕を持たせる

    prompt = (
        f"次の文章を、だいたい {target_chars} 文字くらいの長さで自然に要約してください。\n"
        f"【文章】{text}\n"
        "【要約】"
    )

    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# === 実行部分 ===
text = input("要約したいテキストを貼ってください：\n")
target = int(input("だいたい何文字くらいで要約してほしいですか？："))

print("\n--- 要約結果 ---\n")
result = summarize(text, target_chars=target)
print(result)
print("\n【実際の文字数】：", len(result))